<h2><center>Методы анализа текстов</center></h2>

### Этапы анализа текстов

При обработке текстов выделяют несколько этапов анализа.
* Токенизация (графематический анализ) - выделение абзацев, предложений, токенов. Если абзацы в HTML выделяются довольно просто - по тегам &lt;p&gt;, то с выделением предложений и слов могут быть проблемы.
`Г. Мурманск был основан 3 апреля 1915 г. ниже впадения р. Туломы в Кольский залив. Минимальные IP-адреса: 109.124.97.0 - 109.124.97.3.`
* Морфологический анализ (стемминг, лемматизация) - определение начальной формы слова или его псевдопрефикса, грамматических параметров. Подробнее описан ниже.
* Синтаксический анализ - определение связей между словами (деревья зависимостей) или синтаксически связанных групп слов (деревья составляющих). Первые больше подходят для русского языка, вторые - для английского.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/0/0d/Wearetryingtounderstandthedifference_%282%29.jpg">
    <a href="https://en.wikipedia.org/wiki/Dependency_grammar">Деревья зависимостей</a>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Competing_sentence_diagrams.png/750px-Competing_sentence_diagrams.png">
<a href="https://en.wikipedia.org/wiki/Constituent_(linguistics)">Деревья составляющих</a></center>

* Семантический анализ - определение смысла слова и работа с ним (`за'мок` vs `замо'к`, `удаление ребра связанного графа`; не путать с `он видел их семью своими глазами` где имеет место грамматическая неоднозначность `семья`-`семь`). Последнее время чаще используются дистрибутивные модели языка, о которых мы поговорим на отдельном занятии.

Задачей морфологического анализа является определение начальной формы слова, его части речи и грамматических параметров. В некоторых случаях от слова требуется только начальная форма, в других - только начальная форма и часть речи.<br>
Существует два больших подхода к морфологическому анализу: <b>стемминг</b> и <b>поиск по словарю</b>. Для проведения стемминга оставляется справочник всех окончаний для данного языка. Для пришедшего слова проверяется его окончание и по нему делается прогноз начальной формы и части речи.<br>
Например, мы создаем справочник, в котором записываем все окончания прилагательных: <i>-ому, -ему, -ой, -ая, -ий, -ый, ...</i> Теперь все слова, которые имеют такое окончание будут считаться прилагаельными: <i>синий, циклический, красного, больному</i>. Заодно прилагательными будут считаться причастия (<i>делающий, строившему</i>) и местоимения (<i>мой, твой, твоему</i>). Также не понятно что делать со словами, имеющими пустое окончание. Отдельную проблему составляют такие слова, как <i>стекло, больной, вина</i>, которые могут разбираться несколькими вариантами (это явление называется <b>омонимией</b>). Помимо этого, стеммер может просто откусывать окончания, оставляя лишь псевдооснову.<br>
Большинство проблем здесь решается, но точность работы бессловарных стеммеров находится на уровне 80%. Чтобы повысить точность испольуют морфологический анализ со словарем. Разработчики составляют словарь слов, встретившихся в текстах (<a href="http://opencorpora.org/dict.php">здесь</a> можно найти пример такого словаря). Теперь каждое слово будет искаться в словаре и не предсказываться, а выдаваться точно. Для слов, отсутствующих в словаре, может применяться предсказание, пообное работе стеммера.<br>

## Морфологический анализ

Есть несколько наиболее распространенных библиотеки для морфологического анализа текстов на Python: pymorphy3, pymystem и nltk. Рассмотрим работу с ними.

Библиотеки pymorphy основана на словаре [OpenCorpora](opencorpora.org/) и позволяет проводить анализ отдельных слов, то есть предварительно необходимо провести графематический анализ.

In [1]:
import pymorphy3 # Морфологический анализатор.

Посмотрим на результаты анализа отдельного слова.

In [2]:
morph = pymorphy3.MorphAnalyzer() # Создает объект морфоанализатора и загружет словарь.
wordform = morph.parse('стекло')  # Проведем анализ слова "стекло"...
print(wordform)                 # ... и посмотрим на результат.

[Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='стекло', score=0.690476, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 0),)), Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='стекло', score=0.285714, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 3),)), Parse(word='стекло', tag=OpencorporaTag('VERB,perf,intr neut,sing,past,indc'), normal_form='стечь', score=0.023809, methods_stack=((DictionaryAnalyzer(), 'стекло', 1015, 3),))]


Как видно из вывода, слово "стекло" может быть неодушевленным существительным среднего рода, единственного числа, именительного падежа `tag=OpencorporaTag('NOUN,inan,neut sing,nomn')`, аналогично, но в винительном падеже (`'NOUN,inan,neut sing,accs'`), и глаголом `'VERB,perf,intr neut,sing,past,indc'`. При этом в первой форме оно встречается в 75% случаев (<i>score=0.75</i>), во второй в 18,75% случаев (<i>score=0.1875</i>), а как глагол - лишь в 6,25% (<i>score=0.0625</i>). Самым простым видом борьбы с омонимией является выбор нулевого элемента из списка, возвращенного морфологическим анализом. Такой подход дает около 90% точности при выборе начальной формы и до 80% если мы обращаем внимание на грамматические параметры.

In [3]:
wordform[0].score

0.690476

In [4]:
wordform[0].tag

OpencorporaTag('NOUN,inan,neut sing,nomn')

Pymorphy3 умеет синтезировать нужные нам формы слова. Для этого необходимо получить объект типа `Parse` для нужного слова, а затем вызвать функцию `inflect`.

In [5]:
wordform[0].inflect({'plur','datv'})

Parse(word='стёклам', tag=OpencorporaTag('NOUN,inan,neut plur,datv'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стёклам', 157, 8),))

Помимо этого можно получить вообще всю лексему.

In [6]:
wordform[0].lexeme

[Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 0),)),
 Parse(word='стекла', tag=OpencorporaTag('NOUN,inan,neut sing,gent'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стекла', 157, 1),)),
 Parse(word='стеклу', tag=OpencorporaTag('NOUN,inan,neut sing,datv'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стеклу', 157, 2),)),
 Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стекло', 157, 3),)),
 Parse(word='стеклом', tag=OpencorporaTag('NOUN,inan,neut sing,ablt'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стеклом', 157, 4),)),
 Parse(word='стекле', tag=OpencorporaTag('NOUN,inan,neut sing,loct'), normal_form='стекло', score=1.0, methods_stack=((DictionaryAnalyzer(), 'стекле', 157, 5),)),
 Parse(word='стёкла'

Обратите внимание на `methods_stack=((<DictionaryAnalyzer>, 'лось', 121, 0)` и `methods_stack=((<FakeDictionary>, 'варкалось', 224, 9)`. Наличие строки <FakeDictionary> говорито том, что слово было предсказано. Причем в первом случае оно предсказано как форма слова _лось_, к которому добавлена неизвестная приставка. Во втором случае - это совершенно незнакомое слово.

А ещё pymorphy умеет предсказывать незнакомые слова.

In [7]:
wordform = morph.parse('пырялись') 
wordform

[Parse(word='пырялись', tag=OpencorporaTag('VERB,impf,intr plur,past,indc'), normal_form='пыряться', score=1.0, methods_stack=((FakeDictionary(), 'пырялись', 234, 10), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'ялись')))]

Вместо Pymorphy можно использовать PyMystem. Его плюсом является тот факт, что он сам проводит графематический анализ и снимает омонимию. 

Функция `lemmatize` делит текст на слова и знаки препинания, а затем возвращает для них только начальную форму.

Функция `analyze` возвращает не только начальную форму, но и всю информацию о слове, как это делал перед этим Pymorphy. 

Как видно из примера, делает он это не всегда корректно, но нам не придется думать о том, какое вариант разбора следует взять.

In [8]:
import pymystem3 # Еще один морфологический анализатор. При первом запуске грузит словари из Сети.

In [9]:
mystem = pymystem3.Mystem()
print(mystem.lemmatize('эти типы стали есть в цеху.'))
print(mystem.analyze('эти типы стали есть в цеху.'))

['этот', ' ', 'тип', ' ', 'становиться', ' ', 'есть', ' ', 'в', ' ', 'цех', '.', '\n']
[{'analysis': [{'lex': 'этот', 'wt': 1, 'gr': 'APRO=(им,мн|вин,мн,неод)'}], 'text': 'эти'}, {'text': ' '}, {'analysis': [{'lex': 'тип', 'wt': 0.8700298642, 'gr': 'S,муж,неод=(вин,мн|им,мн)'}], 'text': 'типы'}, {'text': ' '}, {'analysis': [{'lex': 'становиться', 'wt': 0.9821285244, 'gr': 'V,нп=прош,мн,изъяв,сов'}], 'text': 'стали'}, {'text': ' '}, {'analysis': [{'lex': 'есть', 'wt': 0.0492236161, 'gr': 'V,несов,пе=инф'}], 'text': 'есть'}, {'text': ' '}, {'analysis': [{'lex': 'в', 'wt': 0.9999917878, 'gr': 'PR='}], 'text': 'в'}, {'text': ' '}, {'analysis': [{'lex': 'цех', 'wt': 1, 'gr': 'S,муж,неод=(дат,ед|местн,ед)'}], 'text': 'цеху'}, {'text': '.'}, {'text': '\n'}]


Результаты работы можно представить в таком виде.

```
['этот', 
 ' ',
 'тип',
 ' ',
 'становиться',
 ' ',
 'есть',
 ' ',
 'в',
 ' ',
 'цех',
 '.',
 '\n']```
 
```
[
  {'analysis': 
    [{'lex': 'этот', 'wt': 1, 'gr': 'APRO=(им,мн|вин,мн,неод)'}], 
   'text': 'эти'
  }, 
  {
    'text': ' '
  }, 
  {'analysis': 
    [{'lex': 'тип', 'wt': 0.8700298667, 'gr': 'S,муж,неод=(вин,мн|им,мн)'}], 
   'text': 'типы'}, 
  {'text': ' '}, 
  {'analysis': 
    [{'lex': 'становиться', 'wt': 0.9821285009, 'gr': 'V,нп=прош,мн,изъяв,сов'}], 
   'text': 'стали'}, 
  {'text': ' '}, 
  {'analysis': 
    [{'lex': 'есть', 'wt': 0.04922361672, 'gr': 'V,несов,пе=инф'}], 
   'text': 'есть'}, 
  {'text': ' '}, 
  {'analysis': 
    [{'lex': 'в', 'wt': 0.9999917746, 'gr': 'PR='}], 
   'text': 'в'}, 
  {'text': ' '}, 
  {'analysis': 
    [{'lex': 'цех', 'wt': 1, 'gr': 'S,муж,неод=(дат,ед|местн,ед)'}], 
   'text': 'цеху'}, 
  {'text': '.'}, 
  {'text': '\n'}
]
```

То есть результатом является список токенов (в том числе и пробельных или знаков препинания), для части из которых имеется результат анализа, который хранится в словаре с ключём `analysis`. Анализ хранит в списке один или несколько вариантов разбора, у каждого из которых есть лемма `lex`, набор грамматических параметров `gr` и некоторый вес `wt`, который показывает степень уверенности системы в правильности ответа.

In [10]:
my_res=mystem.analyze('эти типы стали есть в цеху.')
if 'analysis' in my_res[0].keys(): # Проверяем, что это не разделитель.
    print(my_res[0]['analysis'][0]['gr'].split("=")[0]) # Берем из него анализ, из того грамматическсие параметы, 
                                                        # а из них выделяем часть речи.

APRO


Еще одна библиотека - NLTK. По сравнению с двумя предыдущими библиотеками она обладает более широкой функциональностью и изначально писалась для работы с разными языками.

In [11]:
import nltk # Иностранный морфологический анализатор.

Перед началом использования необходимо загрузить необходимые библиотеки или корпуса.

In [12]:
nltk.download() # По дороге будут появляться поле ввода. Грузит всё из Сети.

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Можно сразу скачать нужный пакет, если вы знаете как он назыввается.

In [13]:
nltk.download(['averaged_perceptron_tagger_ru', 'stopwords'])

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/edward/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/edward/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Функция `word_tokenize` возвращает начальные формы слов. 

Функция `pos_tag` возвращает список начальных форм и их частей речи.

In [14]:
tokens = nltk.word_tokenize('Эти типы стали есть в цеху') # Токенизация.
bi_tokens = list(nltk.bigrams(tokens))
tokens, bi_tokens

(['Эти', 'типы', 'стали', 'есть', 'в', 'цеху'],
 [('Эти', 'типы'),
  ('типы', 'стали'),
  ('стали', 'есть'),
  ('есть', 'в'),
  ('в', 'цеху')])

In [15]:
pos = nltk.pos_tag(tokens) # Частеречная разметка.
bi_pos = list(nltk.bigrams(pos))
pos, bi_pos


([('Эти', 'JJ'),
  ('типы', 'NNP'),
  ('стали', 'NNP'),
  ('есть', 'NNP'),
  ('в', 'NNP'),
  ('цеху', 'NN')],
 [(('Эти', 'JJ'), ('типы', 'NNP')),
  (('типы', 'NNP'), ('стали', 'NNP')),
  (('стали', 'NNP'), ('есть', 'NNP')),
  (('есть', 'NNP'), ('в', 'NNP')),
  (('в', 'NNP'), ('цеху', 'NN'))])

У NLTK заведен список стоп-слов, которые лучше фильтровать при анализе текстов. Но их не очень много. Зато самые мешающиеся.

In [16]:
# Оставим только те слова, которых нет в списке стоп-слов.
filtered_words = [token for token in tokens if token not in nltk.corpus.stopwords.words('russian')]
print('всего русских стоп-слов', len(nltk.corpus.stopwords.words('russian')))
filtered_words

всего русских стоп-слов 151


['Эти', 'типы', 'стали', 'цеху']

Ту же самую задачу в других библиотеках можно решить при помощи фильтра частей речи. Можно считать, что значимыми являются лишь существительные, прилагательные, глаголы, причастия и деепричастия. Ниже приведены названия частей речи для разных библиотек.
<table>
<tr><th>Часть речи</th><th>Pymorphy</th><th>Mystem</th><th>NLTK</th></tr>
<tr><td>Существительное</td><td>NOUN</td><td>S</td><td>NN</td></tr>
<tr><td>Прилагательное</td><td>ADJF, ADJS</td><td>A</td><td>NNP</td></tr>
<tr><td>Глагол</td><td>VERB</td><td>V</td><td>JJ</td></tr>
<tr><td>Причастие</td><td>PRTF, PRTS</td><td>V</td><td>NNP</td></tr>
<tr><td>Деепричастие</td><td>GRND</td><td>V</td><td>NNP</td></tr>
</table>


Возьмем новость с сайта Лента.ру и приведем все слова её текста к начальным формам при помощи разных библиотек. Прибавим при этом к словам части речи.

In [17]:
import re

In [18]:
text = """
Марсоход «Кьюриосити» отыскал одно из лучших свидетельств существования жидкой воды в кратере Гейл в далеком прошлом Марса. Ровер обнаружил на скалах волнообразную текстуру, которая сформировалась, когда скалы были дном неглубокого озера, сообщается на сайте NASA.\n
«Кьюриосити» работает на Марсе более десяти лет, преодолев за это время почти 30 километров. Основной целью ровера стала пятикилометровая гора Шарпа, расположенная в центре кратера Гейл и покрытая массивом из эродированных слоев осадочных пород. Там он проводит геологические исследования и изучает атмосферу Красной планеты.\n
Восьмого февраля 2023 года команда ровера сообщила, что обнаружила одно из лучших свидетельств существования жидкой воды в кратере за все время работы «Кьюриосити». Поднявшись почти на 800 метров над основанием горы Шарп ровер попал в область, названную «Долиной маркерной полосы» (Marker Band Valley) — тонком слое темных скал, который выделяется на фоне остальной части горы. Породы здесь очень твердые, и ранее ровер не смог получить их образец, хотя сделал несколько попыток.
"""

In [19]:
# Pymorphy
def normalizePymorphy(text):
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    words = []
    for t in tokens:
        pv = morph.parse(t)
        words.append(pv[0].normal_form + '_' + str(pv[0].tag.POS)) # Берем наиболее вероятную форму.
    return words    
        
# Обратите внимание, что про иностранные слова словарь ничего не знает.
normalizePymorphy(text)

['марсоход_NOUN',
 'кьюриосити_NOUN',
 'отыскать_VERB',
 'один_ADJF',
 'из_PREP',
 'хороший_ADJF',
 'свидетельство_NOUN',
 'существование_NOUN',
 'жидкий_ADJF',
 'вода_NOUN',
 'в_PREP',
 'кратер_NOUN',
 'гейл_NOUN',
 'в_PREP',
 'далёкий_ADJF',
 'прошлый_ADJF',
 'марс_NOUN',
 'ровер_NOUN',
 'обнаружить_VERB',
 'на_PREP',
 'скала_NOUN',
 'волнообразный_ADJF',
 'текстура_NOUN',
 'который_ADJF',
 'сформироваться_VERB',
 'когда_CONJ',
 'скала_NOUN',
 'быть_VERB',
 'дно_NOUN',
 'неглубокий_ADJF',
 'озеро_NOUN',
 'сообщаться_VERB',
 'на_PREP',
 'сайт_NOUN',
 'nasa_None',
 'кьюриосити_NOUN',
 'работать_VERB',
 'на_PREP',
 'марс_NOUN',
 'более_ADVB',
 'десять_NUMR',
 'год_NOUN',
 'преодолеть_GRND',
 'за_PREP',
 'это_PRCL',
 'время_NOUN',
 'почти_ADVB',
 'километр_NOUN',
 'основной_ADJF',
 'цель_NOUN',
 'ровер_NOUN',
 'стать_VERB',
 'пятикилометровый_ADJF',
 'гора_NOUN',
 'шарп_NOUN',
 'расположить_PRTF',
 'в_PREP',
 'центр_NOUN',
 'кратер_NOUN',
 'гейл_NOUN',
 'и_CONJ',
 'покрыть_PRTF',
 'масси

In [20]:
# PyMystem
def normalizePymystem(text):
    tokens = mystem.analyze(text)
    words = []
    for t in tokens:
        if 'analysis' in t.keys():
            if t['analysis'] != []:
                words.append(t['analysis'][0]['lex']+'_'+t['analysis'][0]['gr'][0])
            else:
                words.append(t['text']+'_'+'U')
    return words    
        
# Не все считают, что причастие всегда выступает в роли глагола, но иногда так значительно проще.
normalizePymystem(text)

['марсоход_S',
 'кьюриосити_S',
 'отыскивать_V',
 'один_A',
 'из_P',
 'хороший_A',
 'свидетельство_S',
 'существование_S',
 'жидкий_A',
 'вода_S',
 'в_P',
 'кратер_S',
 'гейл_S',
 'в_P',
 'далекий_A',
 'прошлое_S',
 'марс_S',
 'ровер_S',
 'обнаруживать_V',
 'на_P',
 'скала_S',
 'волнообразный_A',
 'текстура_S',
 'который_A',
 'сформировываться_V',
 'когда_C',
 'скала_S',
 'быть_V',
 'дно_S',
 'неглубокий_A',
 'озеро_S',
 'сообщаться_V',
 'на_P',
 'сайт_S',
 'NASA_U',
 'кьюриосити_S',
 'работать_V',
 'на_P',
 'марс_S',
 'много_A',
 'десять_N',
 'год_S',
 'преодолевать_V',
 'за_P',
 'этот_A',
 'время_S',
 'почти_A',
 'километр_S',
 'основной_A',
 'цель_S',
 'ровер_S',
 'становиться_V',
 'пятикилометровый_A',
 'гора_S',
 'шарп_S',
 'располагать_V',
 'в_P',
 'центр_S',
 'кратер_S',
 'гейл_S',
 'и_C',
 'покрывать_V',
 'массив_S',
 'из_P',
 'эродировать_V',
 'слой_S',
 'осадочный_A',
 'порода_S',
 'там_A',
 'он_S',
 'проводить_V',
 'геологический_A',
 'исследование_S',
 'и_C',
 'изучать_V',


In [21]:
# NLTK
def normalizeNLTK(text):
    tokens = nltk.pos_tag(nltk.word_tokenize(text))
    words = []
    for t in tokens:
        if t[0] != t[1]:
            words.append(t[0]+'_'+t[1])
    return words    
        
# А вот здесь с частеречной разметкой всё плохо, а параметров нет вовсе.
normalizeNLTK(text)

['Марсоход_JJ',
 '«_NNP',
 'Кьюриосити_NNP',
 '»_NNP',
 'отыскал_NNP',
 'одно_NNP',
 'из_NNP',
 'лучших_NNP',
 'свидетельств_NNP',
 'существования_NNP',
 'жидкой_NNP',
 'воды_NNP',
 'в_NNP',
 'кратере_NNP',
 'Гейл_NNP',
 'в_NNP',
 'далеком_NNP',
 'прошлом_NNP',
 'Марса_NNP',
 'Ровер_VB',
 'обнаружил_JJ',
 'на_NNP',
 'скалах_NNP',
 'волнообразную_NNP',
 'текстуру_NNP',
 'которая_NNP',
 'сформировалась_NNP',
 'когда_NNP',
 'скалы_NNP',
 'были_NNP',
 'дном_NNP',
 'неглубокого_NNP',
 'озера_NNP',
 'сообщается_NNP',
 'на_NNP',
 'сайте_NNP',
 'NASA_NNP',
 '«_VB',
 'Кьюриосити_JJ',
 '»_NNP',
 'работает_NNP',
 'на_NNP',
 'Марсе_NNP',
 'более_NNP',
 'десяти_NNP',
 'лет_NNP',
 'преодолев_NNP',
 'за_NNP',
 'это_NNP',
 'время_NNP',
 'почти_VBD',
 '30_CD',
 'километров_NN',
 'Основной_CC',
 'целью_JJ',
 'ровера_NNP',
 'стала_NNP',
 'пятикилометровая_NNP',
 'гора_NNP',
 'Шарпа_NNP',
 'расположенная_NNP',
 'в_NNP',
 'центре_NNP',
 'кратера_NNP',
 'Гейл_NNP',
 'и_NNP',
 'покрытая_NNP',
 'массивом_NNP'

In [22]:
from collections import Counter # Не считать же частоты самим.

In [23]:
words = normalizePymorphy(text)
wdict = Counter(words) # Объект сразу посчитает частоты элементов списка.
print(wdict)
print()
print({w:n for w,n in wdict.items() if n>1}) # Посмотрим какие слова встречаются больше одного раза.

Counter({'в_PREP': 5, 'ровер_NOUN': 5, 'на_PREP': 5, 'кьюриосити_NOUN': 3, 'из_PREP': 3, 'кратер_NOUN': 3, 'скала_NOUN': 3, 'гора_NOUN': 3, 'и_CONJ': 3, 'один_ADJF': 2, 'хороший_ADJF': 2, 'свидетельство_NOUN': 2, 'существование_NOUN': 2, 'жидкий_ADJF': 2, 'вода_NOUN': 2, 'гейл_NOUN': 2, 'марс_NOUN': 2, 'обнаружить_VERB': 2, 'который_ADJF': 2, 'год_NOUN': 2, 'за_PREP': 2, 'время_NOUN': 2, 'почти_ADVB': 2, 'шарп_NOUN': 2, 'слой_NOUN': 2, 'порода_NOUN': 2, 'марсоход_NOUN': 1, 'отыскать_VERB': 1, 'далёкий_ADJF': 1, 'прошлый_ADJF': 1, 'волнообразный_ADJF': 1, 'текстура_NOUN': 1, 'сформироваться_VERB': 1, 'когда_CONJ': 1, 'быть_VERB': 1, 'дно_NOUN': 1, 'неглубокий_ADJF': 1, 'озеро_NOUN': 1, 'сообщаться_VERB': 1, 'сайт_NOUN': 1, 'nasa_None': 1, 'работать_VERB': 1, 'более_ADVB': 1, 'десять_NUMR': 1, 'преодолеть_GRND': 1, 'это_PRCL': 1, 'километр_NOUN': 1, 'основной_ADJF': 1, 'цель_NOUN': 1, 'стать_VERB': 1, 'пятикилометровый_ADJF': 1, 'расположить_PRTF': 1, 'центр_NOUN': 1, 'покрыть_PRTF': 1, 

Фактически, выше мы провели преобразование текста в вектор. Пространство вектора определено на словаре текста - количество измерений совпадает с количеством слов, каждому измерению сопоставлено какое-то слово и отложена его частота. Подобный подход называют мешком слов (Bag of Words, BoW), так как все слова перемешиваются, их порядок больше не соблюдается, а сами слова сваливаются в один "мешок".

![](img/donkey_carrot_text.png)

Обратите внимание на распределение частот в отдельных словах и парах. Такое распределение называется [распределением Ципфа](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD_%D0%A6%D0%B8%D0%BF%D1%84%D0%B0) и является характерным практически для любого распределения частот слов и их комбинаций в текстах на любом естественном языке.

$p_i=\frac{p_0}{\beta^{-\alpha*i}}$, где $\alpha\approx1$.

![](https://upload.wikimedia.org/wikipedia/ru/thumb/d/d8/WikipediaZipf20061023.png/450px-WikipediaZipf20061023.png)

Но вообще, для расчета частот существует CountVectorizer, который позволяет сделать это всё за один раз и очень хорошо ложится в конвейер.

Кстати, всё то же самое можно было посчитать при помощи класса [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) из библиотеки [skLearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text). Но её мы оставим на самостоятельное изучение.

А теперь обратите внимание на ещё пару фактов. Во-первых, наиболее частотными оказали служебные слова. Во-вторых, не все слова из аннотации, написанной человеком, вошли в список наиболее частотных.

In [24]:
print({w:n for w,n in wdict.items() if n>1})

{'кьюриосити_NOUN': 3, 'один_ADJF': 2, 'из_PREP': 3, 'хороший_ADJF': 2, 'свидетельство_NOUN': 2, 'существование_NOUN': 2, 'жидкий_ADJF': 2, 'вода_NOUN': 2, 'в_PREP': 5, 'кратер_NOUN': 3, 'гейл_NOUN': 2, 'марс_NOUN': 2, 'ровер_NOUN': 5, 'обнаружить_VERB': 2, 'на_PREP': 5, 'скала_NOUN': 3, 'который_ADJF': 2, 'год_NOUN': 2, 'за_PREP': 2, 'время_NOUN': 2, 'почти_ADVB': 2, 'гора_NOUN': 3, 'шарп_NOUN': 2, 'и_CONJ': 3, 'слой_NOUN': 2, 'порода_NOUN': 2}


Напишем функцию, которая бы оставляла слова только со значимыми частями речи (краткое и полное прилагательное, существительное, глагол, краткая и полная формы причастия, деепричастие).

In [25]:
imp_POS = ['ADJF', 'ADJS', 'NOUN', 'VERB', 'PRTF', 'PRTS', 'GRND']

def getMostFrequentWordsFiltered(text):
    
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    words = []
    for t in tokens:
        pv = morph.parse(t)
        if pv[0].tag.POS in imp_POS and pv[0].normal_form != 'быть':
            words.append(pv[0].normal_form)
    wdict = Counter(words)
    wdict = {w:n for w,n in wdict.items() if n>1}
    return sorted(wdict.items(), key = lambda x: x[1], reverse=True)

In [26]:
getMostFrequentWordsFiltered(text)

[('ровер', 5),
 ('кьюриосити', 3),
 ('кратер', 3),
 ('скала', 3),
 ('гора', 3),
 ('один', 2),
 ('хороший', 2),
 ('свидетельство', 2),
 ('существование', 2),
 ('жидкий', 2),
 ('вода', 2),
 ('гейл', 2),
 ('марс', 2),
 ('обнаружить', 2),
 ('который', 2),
 ('год', 2),
 ('время', 2),
 ('шарп', 2),
 ('слой', 2),
 ('порода', 2)]

Представим себе, что мы взяли много текстов и для каждого из них посчитали вектор слов. Результатом работы является [разреженная матрица](https://docs.scipy.org/doc/scipy/reference/sparse.html) частот слов. Если мы возьмем большое количество текстов, то в кажом из них встречается не так много разных слов, но словарь всех текстов вместе будет огромен. Обработка текстов должна вестись в едином пространстве. Пусть это будет пространство словаря всех текстов (в противном случае у каждого текста будет свое собственное пространство, что чрезвычайно неудобно). Получается, что для текста с маленьким словарем мы должны хранить большое число нулей. Для того, чтобы этого избежать, хранят, например, один раз номер строки, индексы ненулевых значений и сами значения, то есть чуть больше двух чисел на ненулевое значение. Если считать, что словарь заметки - 100 слов, а словарь всех текстов - 100 000 слов, мы получаем экономию места в 500 раз. То, что считалось на кластере, теперь может считаться на недобуке с 2 Гб оперативной памяти.

![](img/term-document-matrix-bow-annotated.png)  
Изображение взято [отсюда](https://livebook.manning.com/book/natural-language-processing-in-action/chapter-4/v-4/61).

Теперь попробуем другой показатель для подсчета важности слов в тексте - $TF*IDF$. Здесь $TF$ - Term Frequency, частота термина в документе, а $IDF$ - Inverted Document Frequency, обратная частота термина в коллекции (количество документов, в которых встречается данный термин).

Идея метрики очень проста. Если слово встречается почти во всех документах - его различительная сила очень мала и само слово не является важным. Если слово часто встречается в данном документе, то оно являетсяя важным для него.


![](img/term-document-matrix-tfidf-annotated.png)  
Изображение взято [отсюда](https://livebook.manning.com/book/natural-language-processing-in-action/chapter-4/v-4/61).


Метрика считается на коллекции документов для каждого слова, каждого документа. Для расчета меры можно использовать `TfidfVectorizer`, который работает так же как `CountVectorizer`.

При помощи функции `fit_transform` можно получить разреженное представление матрицы частот слов. Основная проблема состоит в том, что индексы в матрице представляют собой индексы в словаре переданных текстов. Сам словарь хранится в свойстве `vocabulary_` и умеет возвращать индекс по слову (но не наоборот).

In [27]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

In [28]:
with open('data/war_and_peace3.txt') as fil:
    textWP = fil.read()
# Выделяем все слова написанные русской кириллицей.
words = [w[0] for w in re.findall('([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)', textWP)]
newtext = ' '.join(words)

with open('data/sebastopol.txt') as fil:
    textSb = fil.read()
words3 = [w[0].lower() for w in re.findall('([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)', textSb)]
newtext3 = ' '.join(words3)

In [29]:
counter = CountVectorizer()
# Просим посчитать частоты слов.
res = counter.fit_transform([newtext, newtext3])
# Разреженное представление счетчика.
print(res[0,:10])
# Можно получить индекс по слову, ...
print(counter.vocabulary_.get('левый'))
# ... но не наоборот.
print(counter.vocabulary_.get(20342))

  (0, 6)	3
  (0, 5)	1
  (0, 4)	3
  (0, 9)	1
  (0, 7)	2
  (0, 0)	1
  (0, 8)	1
  (0, 2)	1
  (0, 1)	1
  (0, 3)	1
7400
None


А ещё можно создать функцию-анализатор, которая поможет с обработкой.

In [30]:
def getMeaningfullWords(text: str, morph: pymorphy3.MorphAnalyzer):
    words = []
    tokens = re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)
    for t in tokens:
        pv = morph.parse(t)
        if pv[0].tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTF', 'GRND']:
            words.append(pv[0].normal_form)
    return words

lemmaCounter = CountVectorizer(ngram_range=(1,3), 
                               token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

morph = pymorphy3.MorphAnalyzer()

c = [' '.join(getMeaningfullWords(newtext, morph)),
     ' '.join(getMeaningfullWords(newtext3, morph))]
analyze = lemmaCounter.build_analyzer()
res1 = analyze(c[0])
res2 = lemmaCounter.fit_transform(c)

In [31]:
res1[:10]

['лев',
 'николаевич',
 'толстой',
 'война',
 'мир',
 'тот',
 'олег',
 'колесников',
 'часть',
 'первый']

In [32]:
lemmaCounter = TfidfVectorizer(ngram_range=(1,3), 
                               token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

c = [' '.join(getMeaningfullWords(newtext, morph)),
     ' '.join(getMeaningfullWords(newtext3, morph))]
res_tfidf = lemmaCounter.fit_transform(c)

In [33]:
res_tfidf

<2x124377 sparse matrix of type '<class 'numpy.float64'>'
	with 127153 stored elements in Compressed Sparse Row format>

In [34]:
list(lemmaCounter.vocabulary_.items())[:10]

[('лев', 46014),
 ('николаевич', 59614),
 ('толстой', 109339),
 ('война', 12704),
 ('мир', 50863),
 ('тот', 109674),
 ('олег', 64364),
 ('колесников', 41260),
 ('часть', 119174),
 ('первый', 69467)]

In [35]:
dense_tfidf = res_tfidf.todense()
dense_tfidf[1, -10:]

matrix([[0.00248751, 0.00248751, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.00248751, 0.00248751, 0.00248751]])

In [36]:
tf_dct = dict()
rev_dct = {key: word for word, key in lemmaCounter.vocabulary_.items()}
for i in range(dense_tfidf.shape[1]):
    if dense_tfidf[1, i] != 0:
        tf_dct[rev_dct[i]] = dense_tfidf[1, i]

sorted(tf_dct.items(), key=lambda x: x[1], reverse=True)[:10]

[('быть', 0.4477814550808446),
 ('который', 0.2707927376575859),
 ('сказать', 0.26371318896065554),
 ('володя', 0.24626395219853367),
 ('офицер', 0.1964574763398172),
 ('один', 0.16813928155209582),
 ('брат', 0.14513074828707218),
 ('козелец', 0.12686324810227492),
 ('солдат', 0.11504266632511818),
 ('другой', 0.10973300480242042)]

Но нам необходимо искать новости, которые интересны пользователю.

Для определения меры сходства двух статей теперь может использоваться косинусная мера сходства, рассчитываемая по следующей формуле: $cos(a,b)=\frac{\sum{a_i * b_i}}{\sqrt {\sum{a_i^2}*\sum{b_i^2}}}$.

Косинусная мера сходство смотрит не на близость конечных точек, а на то, в какую сторону направлены векторы.

![](img/cosine_watch.jpg)

Вообще-то, использовать стандартную функцию рассчета косинусной меры сходства из <a href="http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html">sklearn</a> было бы быстрее. Но в данной задаче нам бы пришлось сводить все словари в один, чтобы на одних и тех же местах в векторе были частоты одних и тех же слов. Чтобы избежать подобной работы, напишем собственную функцию рассчета косинусного расстояния, работающую с разреженными векторами в виде питоновских словарей.

Здесь следует остановиться на том, что косинусная мера является мерой расстояния между объектами, причем подобных мер существует достаточно много. Обычно мы используем Евклидово расстояние: $d_E=\sqrt{\sum {(a_i-b_i)^2}}$. Но при обработке текстов оно работает гораздо хуже. Представим себе, что у нас есть два текста: текст статьи и текст статьи, объединенный с самим собой. Евклидово расстояние между ними будет значительным, тогда как содержание не изменится.

Помимо Евклидового расстояния часто используется Манхэттенское расстояние (расстояние городских кварталов). Если взглянуть на карту Манхэттена, то мы увидим, что практически все улицы параллельны и перпендикулярны друг другу. Это означает, что выбирая любой не удлинняющий маршрут из одной точки в другую я пройду одно и то же расстояние: $d_M=\sum {|a_i-b_i|}$. Манхэттенское расстояние используется в тех случаях, когда мы берем, например, взвешенную сумму параметров с тем, чтобы получить единую оценку. Например, оценивая различные офисы мы складываем с некоторыми коэффициентами стоимость, площадь, расстояние от центра или дома, оценку инфраструктуры и так далее. Аналогично можно брать разницу между двумя векторными представлениями офисов, чтобы найти насколько они сходны.

Расстояние Жаккара берет отношение размера пересечения словарей к их объединению: $d_J=\frac{A \cup B}{A \cap B}$. Эта мера проверяет степень совпадения словарей двух текстов. Если словари совпадают полностью, то тексты, скорее всего, говорят об одном и том же.

Однако, в одном тексте может обсуждаться производство шестеренок, а во вводной части однажды будет упомянуто, что они необходимы для сбора механизмов, тогда как в другом тексте будут обсуждаться сами механизмы с единственным упоминанием, что они состоят из шестеренок. Случайное появление отдельных слов сделает тексты более похожими, чем это следует из их содержания. Этот недостаток устраняет косинусная мера сходства: $d_{cos}=\frac{\sum{a_i * b_i}}{\sqrt {\sum{a_i^2}*\sum{b_i^2}}}$. Если в одном из текстов не встречается слово из другого текста, то соответствующий член суммы в числителе будет равен нулю. Если в одном тексте слово встречается часто, а в другом редко, произведение не будет слишком большим. Проблему представляет случай, когда в обоих текстах есть несколько очень часто встречающихся слов. Тогда их произведение будет забивать все остальные слова, искажая общий смысл.

Следует иметь в виду, что косинусная мера сходства является величиной, обратно зависящей от расстояния: два одинаковых текста будут иметь косинусное сходство равное 1, тогда как расстояние между ними равно нулю, и наоборот.

В качестве мер расстояния также используются [корреляция](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D1%80%D1%80%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D1%8F) и [дивергенция Кулльбака-Лейблера](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9A%D1%83%D0%BB%D1%8C%D0%B1%D0%B0%D0%BA%D0%B0_%E2%80%94_%D0%9B%D0%B5%D0%B9%D0%B1%D0%BB%D0%B5%D1%80%D0%B0).
